# Lab Instructions

Find a dataset that interests you. I'd recommend starting on [Kaggle](https://www.kaggle.com/). Read through all of the material about the dataset and download a .CSV file.

1. Write a short summary of the data.  Where did it come from?  How was it collected?  What are the features in the data?  Why is this dataset interesting to you?  

2. Identify 5 interesting questions about your data that you can answer using Pandas methods.  

3. Answer those questions!  You may use any method you want (including LLMs) to help you write your code; however, you should use Pandas to find the answers.  LLMs will not always write code in this way without specific instruction.  

4. Write the answer to your question in a text box underneath the code you used to calculate the answer.




###  ["List of Goods Produced by Child Labor or Forced Labor", US Department of Labor (2024)](https://catalog.data.gov/dataset/list-of-goods-produced-by-child-labor-or-forced-labor-57e0f)

This data is from the US Department of Labor. The primary sources are collected by various institutions and appear to be collected and verified by the Bureau of Internation Labor Affairs, but hosted on the Department of Labor's website [here](https://www.dol.gov/agencies/ilab/research-impact-evaluation).

The dataset features the country/area of the labor, the good being produced, if child labor is being employed, if forced labor is being employed, and if forced child labor is being employed (note: child labor and forced labor do not necessarily imply forced child labor, however forced child labor does necessarily imply both).

I chose this dataset because I have been interested in the field of various kinds of forced labor for some time. In high school I wrote extensively on child soldiers, so I am familiar with sourcing this data and interpreting it. To quote Ayn Rand "A desire presupposes the possibility of action to achieve it; action presupposes a goal worth achieving." Which illustrates the tautological necessity of awareness of such evil if it is to be eliminated. Therefore, I choose to write about it to give another consciousness awareness to it.

Unfortunately, I can only foresee the problem of child labor and child soldiers growing due to recent conflicts, but 2024 is the most recent data that I could find.

### 5 Questions
1. What is the most common good manufactured with child labor?

1. Which country has the most diverse set of goods manufactured by means of the specified labors?

1. Are the most common goods different between child labor, forced labor, or forced child labor?

1. Which country(s) produce a significant amount of child pornography?

1. Is there a single country which produces a significant amount of pornography by means of forced labor, which doesn't also produce a significant amount of child pornography?





In [90]:
import pandas as pd
data_df = pd.read_csv("2024ListofGoodsExcel.csv", encoding='iso-8859-1')

In [91]:
# Most common goods manufactured with child labor
cl_staples = data_df.loc[(data_df['Child Labor'] == True)]['Good'].value_counts()
cl_staples.head(1)

# Get the number of countries utilizing child labor
#len(data_df.loc[(data_df['Child Labor'] == True)]["Country/Area"].unique()) #describe(include='objects')

Good
Gold    26
Name: count, dtype: int64

The most common good manufactured with child labor is gold, with 26 of the 75 countries that use child labor using children to produce gold. This is just over 1/3 of all the countries which use child labor.

In [92]:
# Which country has the most diverse set of goods through these labors

# This implementation looks very silly. To clarify what is happening here:
#   Every record is a pair of Country and Good with associated data about
#   The labor. Therefore, we can simply count the frequency of a country's
#   appearances in the records if we want to count how many unique goods
#   that country has.
data_df.describe(include='object')['Country/Area']['top']

'India'

India produces the most diverse set of goods utilizing a combination of child labor, forced labor, and forced child labor. There are 28 goods listed in the data, only two of which are not obtained through child labor, but only through forced labor of adults, namely tea and shrimp.

In [93]:
# Does the most common good differ amongst the different types of labor
fl_staples = data_df.loc[(data_df['Forced Labor'] == True)]['Good'].value_counts()
fcl_staples = data_df.loc[(data_df['Forced Child Labor'] == True)]['Good'].value_counts()

# bit 1 is a == b
# bit 2 is a == c
# bit 3 is b == c
bit_mask = 0b000
cl_staple = cl_staples.index[0]
fl_staple = fl_staples.index[0]
fcl_staple = fcl_staples.index[0]

print(f"Most common good produced by child labor: {cl_staples.index[0]}")
print(f"Most common good produced by forced labor: {fl_staples.index[0]}")
print(f"Most common good produced by forced child labor: {fcl_staples.index[0]}")

bit_mask += 1 * (cl_staple == fl_staple)
bit_mask += 2 * (cl_staple == fcl_staple)
bit_mask += 4 * (fl_staple == fcl_staple)


match bit_mask:
	case 0b000: print("None of the most common goods are the same amongst these wretched labors.")
	case 0b001: print("The most common good is the same between child labor and forced labor.")
	case 0b010: print("The most common good is the same between child labor and forced child labor.")

	case 0b100: print("The most common good is the same between forced labor and forced child labor.")
	case 0b111: print("The most common good is the same amongst all types of wretched labor.")
	case default: print("Something went gravely wrong somehow...")

Most common good produced by child labor: Gold
Most common good produced by forced labor: Garments
Most common good produced by forced child labor: Bricks
None of the most common goods are the same amongst these wretched labors.


The most common goods produced by each type of labor are all distinct from each other. This implies that the work that children do voluntarily is different from the work they are forced to do, and that there is a difference between what adults are normally forced to do than what children are normally forced to do. This is a narrow interpretation, and there are cases of overlap between jobs that children do (voluntarily or involuntarily) and that adults are forced to do.

In [94]:
# Countries producing significant amount of child pornography
child_porn = data_df.loc[(data_df['Good'] == "Pornography") & (data_df["Child Labor"] == True)]
child_porn

,Country/Area,Good,Child Labor,Forced Labor,Forced Child Labor
144,Colombia,Pornography,1,0,0
291,Mexico,Pornography,1,0,0
361,Paraguay,Pornography,1,0,0
384,Philippines,Pornography,1,0,0
391,Russia,Pornography,1,1,1
418,Thailand,Pornography,1,0,0
447,Ukraine,Pornography,1,0,0


In [ ]:
# Are there any countries that produce pornography with forced labor that do not produce child pornography
forced_porn = data_df.loc[(data_df['Good'] == "Pornography") & (data_df['Forced Labor'] == True)]

# In order to not obscure who is on the list
print(f"A list of all countries which produce forced pornography:")
forced_porn

In [ ]:
fp_countries = forced_porn['Country/Area']
cp_countries = child_porn['Country/Area']

# We want to see if any two points of data do not match
found_match = False
for x in fp_countries:
	found_match = False
	for y in cp_countries:
		if x == y:
			found_match = True
			break
	if not found_match:
		break


if found_match & (len(forced_porn) != 0) & (len(child_porn) != 0): # This will only trigger if every point in forced_porn mapped to a point in child_porn
	print("Every country which produces pornography via forced labor, produces a significant amount of child pornography")
else:
	print("There is at least one country that forces the production of pornography, which doesn't also produce child pornography")

Every country which produces pornography via forced labor, produces a significant amount of child pornography. In the data we are given, the only country which produces a significant amount of pornography via forced labor, Russia, also does so through forced child labor. It is sickening, and unclear whether there are adults who are forced into making pornography as well.

There is not enough data given to draw conclusive results from, but there may be a relation between areas that impose upon adults to film pornography, and areas which also force children to make porn.